In [1]:
!pip install langchain langchain-community transformers vllm faiss-gpu pypdf sentence-transformers huggingface_hub


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
# 필요한 라이브러리 import
import os
import tempfile
import requests
from urllib.request import urlretrieve

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

INFO 03-29 00:41:27 [__init__.py:239] Automatically detected platform cuda.


In [3]:
# PDF 파일 다운로드
pdf_url = "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_usa_2024.pdf"
temp_dir = tempfile.gettempdir()
pdf_path = os.path.join(temp_dir, "ict_usa_2024.pdf")

print(f"다운로드 중: {pdf_url}")
urlretrieve(pdf_url, pdf_path)
print(f"PDF가 다운로드 되었습니다: {pdf_path}")

다운로드 중: https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_usa_2024.pdf
PDF가 다운로드 되었습니다: /tmp/ict_usa_2024.pdf


In [4]:
# PDF 로드
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"PDF에서 추출된 문서 수: {len(documents)}")
print(f"첫 번째 문서 샘플: {documents[0].page_content[:200]}")

PDF에서 추출된 문서 수: 29
첫 번째 문서 샘플: 미국


In [5]:
# 텍스트 청킹
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
chunks = text_splitter.split_documents(documents)
print(f"청킹 전 청크 수: {len(chunks)}")

청킹 전 청크 수: 46


In [6]:
# 길이가 10 이하인 문서 필터링
filtered_chunks = [chunk for chunk in chunks if len(chunk.page_content) > 10]
print(f"필터링 전 청크 수: {len(chunks)}")
print(f"필터링 후 청크 수: {len(filtered_chunks)}")

# 필터링된 청크가 있으면 제거된 내용 샘플 출력
if len(chunks) != len(filtered_chunks):
    removed = [chunk for chunk in chunks if len(chunk.page_content) <= 10]
    print(f"제거된 청크 샘플 ({len(removed)}개): {removed}")
    print('--')

chunks = filtered_chunks
print(f"첫 번째 청크 샘플: {chunks[0].page_content}")

필터링 전 청크 수: 46
필터링 후 청크 수: 43
제거된 청크 샘플 (3개): [Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 11.0.0.2129', 'creationdate': '2024-06-04T14:47:21+09:00', 'author': 'ec21rnc', 'moddate': '2024-06-04T14:47:21+09:00', 'pdfversion': '1.4', 'source': '/tmp/ict_usa_2024.pdf', 'total_pages': 29, 'page': 0, 'page_label': '1'}, page_content='미국'), Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 11.0.0.2129', 'creationdate': '2024-06-04T14:47:21+09:00', 'author': 'ec21rnc', 'moddate': '2024-06-04T14:47:21+09:00', 'pdfversion': '1.4', 'source': '/tmp/ict_usa_2024.pdf', 'total_pages': 29, 'page': 3, 'page_label': '4'}, page_content='4'), Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 11.0.0.2129', 'creationdate': '2024-06-04T14:47:21+09:00', 'author': 'ec21rnc', 'moddate': '2024-06-04T14:47:21+09:00', 'pdfversion': '1.4', 'source': '/tmp/ict_usa_2024.pdf', 'total_pages': 29, 'page': 16, 'page_label': '17'}, page_c

In [7]:
# 임베딩 모델 설정 (BAAI/bge-m3 사용, GPU 사용)
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'} # GPU 사용
)

/tmp/ipykernel_1834/1297578870.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [8]:
# 벡터 DB 생성 (FAISS GPU 버전 사용)
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f"벡터 DB가 생성되었습니다.")

벡터 DB가 생성되었습니다.


In [9]:
# 검색 테스트
query = "미국의 ICT 정책의 주요 내용은 무엇인가요?"
docs = vectorstore.similarity_search(query, k=3)
print(f"검색된 문서 수: {len(docs)}")
print(f"첫 번째 검색 결과: {docs[0].page_content[:200]}")

검색된 문서 수: 3
첫 번째 검색 결과: 9 
Ⅰ. ICT 국가 산업 현황 3.ICT 주요정책  ① 국가 스펙트럼 전략(National Spectrum Strategy)
 미국 신규 주파수 공급 전략• 2023년 11월 조 바이든(Joe Biden) 행정부는 2,700MHz 이상의 신규 주파수 공급 전략인 ‘국가 스펙트럼 전략(National Spectrum Strategy)’을 발표함. 또 명확


In [12]:
# 원본 VLLM 라이브러리 직접 사용
# VLLM 모델 설정
vllm_model = LLM(
    model="iamjoon/llama-3-8b-rag-ko-checkpoint-285",
    dtype="bfloat16",
)

WARNING 03-29 00:44:26 [config.py:2614] Casting torch.float16 to torch.bfloat16.
INFO 03-29 00:44:26 [config.py:585] This model supports multiple tasks: {'embed', 'reward', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
INFO 03-29 00:44:26 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 03-29 00:44:31 [__init__.py:239] Automatically detected platform cuda.
INFO 03-29 00:44:32 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='iamjoon/llama-3-8b-rag-ko-checkpoint-285', speculative_config=None, tokenizer='iamjoon/llama-3-8b-rag-ko-checkpoint-285', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cu

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.33it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:11<00:13,  6.61s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:22<00:08,  8.80s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:32<00:00,  9.12s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:32<00:00,  8.12s/it]



INFO 03-29 00:45:07 [loader.py:447] Loading weights took 32.65 seconds
INFO 03-29 00:45:08 [gpu_model_runner.py:1186] Model loading took 14.9596 GB and 33.986937 seconds
INFO 03-29 00:45:14 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/fe41fc8ecb/rank_0_0 for vLLM's torch.compile
INFO 03-29 00:45:14 [backends.py:425] Dynamo bytecode transform time: 6.36 s
INFO 03-29 00:45:17 [backends.py:132] Cache the graph of shape None for later use
INFO 03-29 00:45:40 [backends.py:144] Compiling a graph for general shape takes 25.36 s
INFO 03-29 00:45:53 [monitor.py:33] torch.compile takes 31.72 s in total
INFO 03-29 00:45:53 [kv_cache_utils.py:566] GPU KV cache size: 387,520 tokens
INFO 03-29 00:45:53 [kv_cache_utils.py:569] Maximum concurrency for 8,192 tokens per request: 47.30x
INFO 03-29 00:46:15 [gpu_model_runner.py:1534] Graph capturing finished in 22 secs, took 0.52 GiB
INFO 03-29 00:46:15 [core.py:151] init engine (profile, create kv cache, warmup model) to

In [13]:
# 생성 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
)

In [14]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("iamjoon/llama-3-8b-rag-ko-checkpoint-285")
print(f"모델의 기본 chat template 확인: {tokenizer.chat_template}")

모델의 기본 chat template 확인: {% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>

'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>

' }}


In [15]:
# apply_chat_template 예시 출력
example_messages = [
    {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
    {"role": "user", "content": "안녕하세요, 도와주세요."}
]
formatted_example = tokenizer.apply_chat_template(
    example_messages,
    tokenize=False,
    add_generation_prompt=True
)
print("\napply_chat_template 예시:")
print(formatted_example)


apply_chat_template 예시:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 도움이 되는 AI 어시스턴트입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

안녕하세요, 도와주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [16]:
# RAG 프롬프트 템플릿 정의
system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
{search_result}"""

In [17]:
# 검색 결과 포맷팅 함수
def format_docs(docs):
    formatted_results = []
    for i, doc in enumerate(docs):
        formatted_results.append(f"문서{i+1}: {doc.page_content}")
    return "\n-----\n".join(formatted_results)

In [18]:
# 원본 VLLM을 사용한 RAG 함수 구현
def generate_rag_response(question, retriever, tokenizer, vllm_model, sampling_params):
    """
    검색 결과와 질문을 기반으로 RAG 응답을 생성하는 함수
    """
    # 검색 수행
    retrieved_docs = retriever.invoke(question)
    
    # 검색 결과 포맷팅
    formatted_results = format_docs(retrieved_docs)
    
    # 메시지 준비
    formatted_system = system_message.format(search_result=formatted_results)
    messages = [
        {"role": "system", "content": formatted_system},
        {"role": "user", "content": question}
    ]
    
    # 토크나이저로 프롬프트 포맷팅
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # VLLM으로 생성
    outputs = vllm_model.generate([prompt], sampling_params)
    
    # 응답 추출 및 반환
    response = outputs[0].outputs[0].text
    return response, formatted_results, retrieved_docs

## 답변 순서 이해하기

### 질문

In [34]:
# 원본 VLLM을 사용한 RAG 실행 예시
question = "미국의 ICT 정책 중 AI 관련 주요 내용은 무엇인가?"
print("질문:", question)

질문: 미국의 ICT 정책 중 AI 관련 주요 내용은 무엇인가?


### 검색기 설정 및 검색 수행

In [35]:
# 검색기 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [36]:
# 1. 검색 수행
retrieved_docs = retriever.invoke(question)
print(f"검색된 문서 수: {len(retrieved_docs)}")

검색된 문서 수: 3


### 포맷팅

In [37]:
# 2. 검색 결과 포맷팅
formatted_results = format_docs(retrieved_docs)
print("\n검색 결과 (포맷팅):")
print(formatted_results)


검색 결과 (포맷팅):
문서1: 12 
Ⅰ. ICT 국가 산업 현황 4.ICT 주요법령 및 규제  ① AI 행정 명령
 AI 위험 관리를 위한 행정 명령• 2023년 11월 미국 정부는 AI 관련 위험을 관리하기 위한 ‘안전하고 신뢰할 수 있는 AI의 개발 및 활용에 관한 행정명령’을 발표함[표 7] AI 행정명령 주요 내용구분 주요 내용
AI 안전 및 보안에 대한 새로운 표준 수립
‣ 국방물자생산법에 따라 국가 안보, 경제안보, 공중보건 및 안전에 위험을 초래할 수 있는 기반 모델 개발 기업은 AI 시스템 안전 테스트 결과와 기타 중요한 정보를 미국 정부에 공유‣  AI 시스템의 안전성, 보안, 신뢰성을 확인하는 데 도움이 되는 표준, 도구, 테스트 개발‣ 생물학적 합성 스크리닝에 대한 강력한 기준 개발‣ AI가 생성한 콘텐츠를 탐지하고 공식 콘텐츠를 인증하기 위한 표준과 모범사례를 확립하여, AI를 이용한 사기 및 속임수로부터 국민을 보호‣ 중요한 소프트웨어의 취약점을 찾아 수정할 수 있는 AI 툴 개발을 위한 프로그램 수립‣ AI와 보안에 대한 추가 조치를 지시하는 국가안보각서 개발
미국인의 개인정보 보호‣ 개인정보보호 기술의 개발과 활용을 가속화 하기 위한 연방정부의 지원을 우선시‣ 암호화 도구 등 개인정보보호 연구 및 기술 강화‣ 기관이 상업적 용도의 정보 수집 및 이용방법 평가, AI위험 고려 제고 위한 개인정보 지침 강화‣ 연방기관이 개인정보보호 기술의 효과를 평가할 수 있는 지침 개발형평성 및 시민권 증진‣ AI 알고리즘이 처벌을 유도하지 않도록 연방급여 프로그램, 연방 계약자에게 명확한 지침을 제공‣ AI와 관련된 민권 침해조사 및 모범사례 교육 등을 통해 알고리즘의 차별 해소‣ 범죄 예측, 포렌식, 위험성 평가 등 AI 사용 모범사례를 개발, 형사 사법제도 전반의 공정성 보장소비자, 환자, 학생을 위한 지원‣ 의료 분야에서 AI의 책임감 있는 사용 촉진, AI 관련 의료 분야 위험 보고 체계 수립‣ AI 지원 교육 도구 배포 지원
-----

### 메시지 포맷팅

In [38]:
# 3. 메시지 준비
formatted_system = system_message.format(search_result=formatted_results)
messages = [
    {"role": "system", "content": formatted_system},
    {"role": "user", "content": question}
]
print("\n메시지 형식:")
print(f"System: {messages[0]['content']}")
print(f"User: {messages[1]['content']}")


메시지 형식:
System: 당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 12 
Ⅰ. ICT 국가 산업 현황 4.ICT 주요법령 및 규제  ① AI 행정 명령
 AI 위험 관리를 위한 행정 명령• 2023년 11월 미국 정부는 AI 관련 위험을 관리하기 위한 ‘안전하고 신뢰할 수 있는 AI의 개발 및 활용에 관한 행정명령’을 발표함[표 7] AI 행정명령 주요 내용구분 주요 내용
AI 안전 및 보안에 대한 새로운 표준 수립
‣ 국방물자생산법에 따라 국가 안보, 경제안보, 공중보건 및 안전에 위험을 초래할 수 있는 기반 모델 개발 기업은 AI 시스템 안전 테스트 결과와 기타 중요한 정보를 미국 정부에 공유‣  AI 시스템의 안전성, 보안, 신뢰성을 확인하는 데 도움이 되는 표준, 도구, 테스트 개발‣ 생물학적 합성 스크리닝에 대한 강력한 기준 개발‣ AI가 생성한 콘텐츠를 탐지하고 공식 콘텐츠를 인증하기 위한 표준과 모범사례를 확립하여, AI를 이용한 사기 및 속임수로부터 국민을 보호‣ 중요한 소프트웨어의 취약점을 찾아 수정할 수 있는 AI 툴 개발을 위한 프로그램 수립‣ AI와 보안에 대한 추가 조치를 지시하는 국가안보각서 개발
미국인의 개인정보 보호‣ 개

### 챗 템플릿

In [39]:
# 4. 토크나이저로 프롬프트 포맷팅
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print("\n최종 프롬프트:")
print(prompt)


최종 프롬프트:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 12 
Ⅰ. ICT 국가 산업 현황 4.ICT 주요법령 및 규제  ① AI 행정 명령
 AI 위험 관리를 위한 행정 명령• 2023년 11월 미국 정부는 AI 관련 위험을 관리하기 위한 ‘안전하고 신뢰할 수 있는 AI의 개발 및 활용에 관한 행정명령’을 발표함[표 7] AI 행정명령 주요 내용구분 주요 내용
AI 안전 및 보안에 대한 새로운 표준 수립
‣ 국방물자생산법에 따라 국가 안보, 경제안보, 공중보건 및 안전에 위험을 초래할 수 있는 기반 모델 개발 기업은 AI 시스템 안전 테스트 결과와 기타 중요한 정보를 미국 정부에 공유‣  AI 시스템의 안전성, 보안, 신뢰성을 확인하는 데 도움이 되는 표준, 도구, 테스트 개발‣ 생물학적 합성 스크리닝에 대한 강력한 기준 개발‣ AI가 생성한 콘텐츠를 탐지하고 공식 콘텐츠를 인증하기 위한 표준과 모범사례를 확립하여, AI를 이용한 사기 및 속임수로부터 국민을 보호‣ 중요한 소프트웨어의 취약점을 찾아 수정할 수 있는 AI 툴 개발을 위한 프로그

### vLLM을 이용한 최종 답변

In [40]:
# 5. VLLM으로 생성
outputs = vllm_model.generate([prompt], sampling_params)
response = outputs[0].outputs[0].text

# 6. 최종 답변 출력
print("\n최종 답변:")
print(response)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.15s/it, est. speed input: 305.82 toks/s, output: 82.82 toks/s]


최종 답변:
미국의 ICT 정책 중 AI 관련 주요 내용은 다음과 같습니다.

1. **AI 행정 명령 (AI Safety and Security Executive Order)**: 2023년 11월, 미국 정부는 AI 관련 위험을 관리하기 위한 행정 명령을 발표했습니다. 이 명령은 AI 시스템의 안전성, 보안, 신뢰성을 확인하는 표준, 도구, 테스트 개발을 포함합니다. 또한, AI가 생성한 콘텐츠를 탐지하고 공식 콘텐츠를 인증하기 위한 표준과 모범사례를 확립하고, 중요한 소프트웨어의 취약점을 찾아 수정할 수 있는 AI 툴 개발을 위한 프로그램을 수립합니다. 이 명령은 국가 안보, 경제안보, 공중보건 및 안전에 위험을 초래할 수 있는 기반 모델 개발 기업이 AI 시스템 안전 테스트 결과와 기타 중요한 정보를 미국 정부에 공유하도록 요구합니다[[ref1]].

2. **AI 안전 서약 (Summit on AI Safety)**: 2023년 7월, 조 바이든 대통령은 AI 기술을 개발하는 주요 기업이 새로운 기술과 시스템을 설계, 공유, 테스트하는 연구개발 과정에 필요한 3가지 원칙과 8가지 세부 조항으로 이루어진 AI 안전 서약을 발표했습니다. 이 서약은 안전, 보안, 신뢰 등 3가지 원칙을 기반으로 책임감 있는 AI 개발을 추진하는 것을 목적으로 합니다. 주요 기업 7곳이 이 서약에 동의했으며, 미국 행정부는 자국에서 AI 안전 서약 의제를 진전시키고, 동맹국 및 파트너와 협력하여 AI 개발 및 사용을 관리하기 위한 국제 프레임워크를 구축할 예정입니다[[ref2]].

이 두 가지 정책은 AI의 안전성과 보안성을 강화하고, AI의 책임감 있는 개발과 사용을 촉진하는 데 중점을 두고 있습니다.


### 함수를 사용하여 한 번에 처리하기

In [42]:
# 다른 질문으로 테스트 - 함수 사용
question = "미국의 디지털 인프라 투자에 대해 설명해주세요."
print("\n질문:", question)

# 1. 함수 호출하여 RAG 실행
response, formatted_results, retrieved_docs = generate_rag_response(
    question=question,
    retriever=retriever,
    tokenizer=tokenizer,
    vllm_model=vllm_model,
    sampling_params=sampling_params
)

# 2. 결과 출력
print(f"검색된 문서 수: {len(retrieved_docs)}")
print("\n검색 결과:")
print(formatted_results)
print("===")
print("\n최종 답변:")
print(response)



질문: 미국의 디지털 인프라 투자에 대해 설명해주세요.


Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.60s/it, est. speed input: 287.15 toks/s, output: 81.78 toks/s]

검색된 문서 수: 3

검색 결과:
문서1: 5 
Ⅰ. ICT 국가 산업 현황 1.국가 개황
 미국, 글로벌 혁신지수 세계 3위• 미국 인터넷 사용자 비중은 97.1%이며 고정 광대역 가입자 비중은 37.8%임• 미국 글로벌 혁신지수는 세계 3위임. ‘지식 및 기술 생산’이 상대적으로 우위에 있으며, ‘인프라’가 상대적 열위를 보임
 미국 경제 성장세 양호, 기준 금리 인하에는 신중 • 미국 상무부는 2023년 4분기 성장률이 3.2%로 집계됐다고 발표함. 이는 고금리 기조를 감안할 때 양호한 편이라는 평가임• 2024년 1월 소비자 물가는 예측치(2.9%)보다 높은 3.1%를 기록하여 연준 고위인사들은 기준금리 인하에 대해 신중한 입장을 표명함[표 1] 미국 ICT 국가 개황구분 기준년도한국 미국
일반개황
인구 20235,150만 5,000명3억 3,669만 명면적(1,000ha)202310,043983,151언어 2023한국어영어, 스페인어종교 2023기독교, 불교 등 개신교, 가톨릭 등정부형태 / 의회제도2023대통령제 / 단원제대통령제 / 양원제
경제현황GDP 20231.70조 달러27.97조 달러경제 성장률20231.31%1.5%S&P 신용등급2023AA AA+
ICT현황이동전화 가입자 수 2022149명(100명당) 110명(100명당)이동통신 가입자 수 2022120명(100명당) 174명(100명당)유선 브로드밴드 가입자 수 202245명(100명당) 38명(100명당)인터넷 사용자 비중202297%97%
글로벌혁신지수
총점 (순위) 202358.6 (10)63.5 (3)인적자원 및 연구202366.9 (1) 56.5 (12)인프라202360.6 (11)56.7 (25)지식 및 기술 생산202353.3 (11)63.7 (2)창조적 생산 202358.2 (5) 53.0 (12)출처 : World Bank, IMF, 국제전기통신연합(ITU), 세계지적재산권기구(WIPO)
-----
문서2: 미국 근로자를 위한 일자리 및 인력 파이프라인 창출
‣ 반도